<a href="https://colab.research.google.com/github/zman2013/tts/blob/main/tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install all required dependencies
!pip install pytube requests git+https://github.com/openai/whisper.git transformers torch torchaudio unidecode
!apt-get install espeak
!pip install git+https://github.com/pytorch/audio.git

# Clone MeloTTS repository (not executable in Python cell, run in a separate shell cell)
!git clone https://github.com/myshell-ai/MeloTTS.git
%cd /content/MeloTTS
!pip install -e .
!python -m unidic download

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-14_dorhx
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-14_dorhx
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802825 sha256=800ec36f4ca3f53c68b771a9c17dfce10ff952e2a5a0805cc762aaa377789ec6
  Stored in directory: /tmp/pip-ephem-wheel-cache-uexqff92/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-w

In [4]:
# Import necessary libraries
import os
import subprocess
from pytube import YouTube
import whisper
from IPython.display import Audio

# Function to download YouTube video
def download_youtube_video(url, save_path='./'):
    yt = YouTube(url)
    video = yt.streams.get_highest_resolution()
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    video_file_path = video.download(save_path)
    print(f"Video saved to: {video_file_path}")
    return video_file_path

# Function to extract audio from video
def extract_audio_from_video(video_file_path, audio_format="mp3"):
    base, _ = os.path.splitext(video_file_path)
    audio_file_path = f"{base}.{audio_format}"
    command = ['ffmpeg', '-i', video_file_path, '-q:a', '0', '-map', 'a', audio_file_path, '-y']
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(f"Audio extracted to: {audio_file_path}")
    return audio_file_path

# Function to transcribe audio to text with Whisper
def audio_to_text_whisper_with_timestamps(audio_file_path):
    model = whisper.load_model("small")
    result = model.transcribe(audio_file_path)
    segments = result.get("segments", [])

    # 拼接带有时间戳的文本，时间戳保留一位小数
    text_with_timestamps = ""
    for segment in segments:
        start = segment.get("start")
        end = segment.get("end")
        text = segment.get("text")
        # 使用格式化字符串设置时间戳的格式
        text_with_timestamps += f"{start:.1f}-{end:.1f}: {text}\n"

    return text_with_timestamps

# Example usage
url = "https://www.youtube.com/watch?v=5OSP5DNAozU"
video_file_path = download_youtube_video(url)
audio_file_path = extract_audio_from_video(video_file_path)
transcribed_text = audio_to_text_whisper_with_timestamps(audio_file_path)
print(f'transcribed_text: {transcribed_text}')

def merge_sentences(input_text):
    lines = input_text.strip().split("\n")
    merged_lines = []
    current_text = ""
    start_time = ""
    end_time = ""

    for line in lines:
        if not line:
            continue  # 跳过空行
        parts = line.split(": ", 1)
        if len(parts) != 2:
            continue  # 忽略格式不正确的行
        time_range, sentence = parts
        start, end = time_range.split('-')

        if sentence.endswith(('.', '?', '!')):
            # 如果当前行以结束符结尾
            if current_text:
                # 如果当前文本非空，则先添加当前文本到结果中
                merged_lines.append(f"{start_time}-{end}: {current_text} {sentence}")
                current_text = ""
            else:
                merged_lines.append(line)  # 添加当前行到结果中
        else:
            # 如果当前行不以结束符结尾，需要合并
            if not current_text:
                # 如果当前文本为空，这是合并序列的开始
                start_time = start
                current_text = sentence
            else:
                # 否则，继续合并到当前文本
                current_text += " " + sentence
            end_time = end  # 更新结束时间

    # 确保最后一段合并的文本被添加到结果中
    if current_text:
        merged_lines.append(f"{start_time}-{end_time}: {current_text}")

    return "\n".join(merged_lines)

merged_transcribed_text = merge_sentences(transcribed_text)
print(f'merged transcribed_text: {merged_transcribed_text}')

Video saved to: /content/MeloTTS/./Building domain-specific compilers quickly with MLIR compiler infrastructure  Chris Lattner.mp4
Audio extracted to: /content/MeloTTS/./Building domain-specific compilers quickly with MLIR compiler infrastructure  Chris Lattner.mp3
transcribed_text: 0.0-5.4:  We built this new compiler framework called MLIR.
5.4-5.9:  Yes.
5.9-8.4:  MLIR is a whole new framework.
8.4-11.4:  It's not many people think it's about machine learning.
11.4-14.6:  The ML stands for multi-level because compiler people
14.6-16.1:  can't name things very well, I guess.
16.1-19.5:  Can we dig into what MLIR is?
19.5-22.1:  Yeah. So when you look at compilers,
22.1-26.0:  compilers have historically been solutions for a given space.
26.0-31.8:  So LLVM is really good for dealing with CPUs, let's just say,
31.8-32.5:  at a high level.
32.5-34.4:  You look at Java.
34.4-36.0:  Java has a JVM.
36.0-38.6:  The JVM is very good for garbage collected languages
38.6-39.9:  that need dyna

In [5]:
import os
import torch
from pydub import AudioSegment
from IPython.display import Audio
import hashlib
from melo.api import TTS
from pydub import AudioSegment


# 假设的文本内容
transcribed_text = """
0.0-5.4: 我们开发了一种全新的编译器框架,称为 MLIR(Multi-Level Intermediate Representation)。
5.4-5.9: 是的。
5.9-8.4: MLIR 是一个全新的编译器基础架构。
8.4-11.4: 这与很多人误解它与机器学习有关不同。
11.4-16.1: 其中 ML 代表多级(Multi-Level),因为编译器人员在命名方面可能不太擅长。
16.1-19.5: 能否深入解释一下 MLIR 是什么?
19.5-26.0: 好的。对于编译器而言, 传统上它们都是为特定领域量身定制的解决方案。
"""

# Function to convert text to speech with MeloTTS-Chinese
# 修改后的 text_to_speech_melotts 函数
def text_to_speech_melotts(text, audio_file_path, target_duration):
    initial_speed = 1.0  # 初始速度设定为1.0
    device = "cuda:0" if torch.cuda.is_available() else "cpu"

    model = TTS(language='ZH', device=device)
    speaker_ids = model.hps.data.spk2id

    # 首次尝试以初始速度生成语音
    model.tts_to_file(text, speaker_ids['ZH'], audio_file_path, speed=initial_speed)

    # 计算生成的语音文件的时长
    generated_audio = AudioSegment.from_file(audio_file_path)
    generated_duration = len(generated_audio) / 1000.0  # 将毫秒转换为秒

    # 如果生成的语音时长不等于目标时长，则调整速度
    if generated_duration != target_duration:
        adjusted_speed = initial_speed * (generated_duration / target_duration)
        # 使用调整后的速度重新生成语音，以匹配目标时长
        model.tts_to_file(text, speaker_ids['ZH'], audio_file_path, speed=adjusted_speed)

    return audio_file_path

# 步骤1: 解析文本
def parse_text(text):
    lines = text.strip().split('\n')
    segments = []
    for line in lines:
        time_range, content = line.split(': ', 1)
        begin_time, end_time = map(float, time_range.split('-'))
        segments.append((begin_time, end_time, content))
    return segments

# 修改后的步骤2: 文本到语音转换，为每个文件生成唯一名称
def generate_tts_audio(segments, output_dir):
    tts_audio_segments = []
    os.makedirs(output_dir, exist_ok=True)

    for begin_time, end_time, content in segments:
        unique_filename = f"{begin_time}-{end_time}.mp3"
        unique_filepath = os.path.join(output_dir, unique_filename)
        target_duration = end_time - begin_time

        # 调用修改后的 TTS 函数，传递内容、文件路径及目标时长
        audio_path = text_to_speech_melotts(content, unique_filepath, target_duration)
        tts_audio_segments.append((begin_time, end_time, audio_path))

    return tts_audio_segments

# 步骤3: 拼接音频
def concatenate_audio_segments(tts_audio_segments):
    combined = AudioSegment.silent(duration=0)  # 创建一个初始的静音片段
    for _, _, audio_path in tts_audio_segments:
        segment = AudioSegment.from_file(audio_path)
        combined += segment
    return combined

# 步骤4: 保存并播放音频
def save_and_play_audio(combined_audio, output_path="combined_tts_audio.mp3"):
    combined_audio.export(output_path, format="mp3")
    return Audio(output_path)

# 执行流程
tts_output_dir = '/content/tts_output_dir'
segments = parse_text(transcribed_text)
tts_audio_segments = generate_tts_audio(segments, tts_output_dir)
combined_audio = concatenate_audio_segments(tts_audio_segments)
audio_player = save_and_play_audio(combined_audio)

# 显示播放器
audio_player


tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

100%|██████████| 1/1 [00:00<00:00,  4.68it/s]


 > Text split to sentences.
好的. 对于编译器而言,
传统上它们都是为特定领域量身定制的解决方案.
 > ===========================


100%|██████████| 2/2 [00:00<00:00,  5.72it/s]


 > Text split to sentences.
好的. 对于编译器而言,
传统上它们都是为特定领域量身定制的解决方案.
 > ===========================


100%|██████████| 2/2 [00:00<00:00,  5.58it/s]


In [ ]:
from pydub import AudioSegment
import hashlib
import os

origin_audio_dir = '/content/origin_audio_dir'
def split_audio_segments(audio_file_path, segments, output_dir):
    # 加载原始音频文件
    audio = AudioSegment.from_file(audio_file_path)

    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)

    split_audio_segments = []
    for begin_time, end_time, content in segments:
        unique_filename = f"{begin_time}-{end_time}.mp3"
        unique_filepath = os.path.join(output_dir, unique_filename)

        # 计算切分时间点，pydub中时间单位为毫秒
        start_ms = begin_time * 1000
        end_ms = end_time * 1000

        # 切分音频
        segment_audio = audio[start_ms:end_ms]

        # 导出切分后的音频段
        segment_audio.export(unique_filepath, format="mp3")
        split_audio_segments.append((begin_time, end_time, unique_filepath))

    return split_audio_segments

origin_audio_segments=split_audio_segments(audio_file_path, tts_audio_segments, origin_audio_dir)

In [ ]:
!git clone https://github.com/myshell-ai/OpenVoice.git
%cd /content/OpenVoice
!ls
!pip install -e .
!wget https://myshell-public-repo-hosting.s3.amazonaws.com/openvoice/checkpoints_1226.zip
!unzip checkpoints_1226.zip -d .

Cloning into 'OpenVoice'...
remote: Enumerating objects: 376, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 376 (delta 132), reused 126 (delta 98), pack-reused 183
Receiving objects: 100% (376/376), 2.92 MiB | 19.42 MiB/s, done.
Resolving deltas: 100% (185/185), done.
/content/OpenVoice
demo_part1.ipynb  docs	   openvoice  requirements.txt	setup.py
demo_part2.ipynb  LICENSE  README.md  resources
Obtaining file:///content/OpenVoice
  Preparing metadata (setup.py) ... done
  Running setup.py develop for MyShell-OpenVoice
--2024-03-10 14:02:16--  https://myshell-public-repo-hosting.s3.amazonaws.com/openvoice/checkpoints_1226.zip
Resolving myshell-public-repo-hosting.s3.amazonaws.com (myshell-public-repo-hosting.s3.amazonaws.com)... 3.5.10.188, 52.216.211.233, 52.217.71.172, ...
Connecting to myshell-public-repo-hosting.s3.amazonaws.com (myshell-public-repo-hosting.s3.amazonaws.com)|3.5.10.188|:443... connected.
HTTP re

In [ ]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

# 假设的检查点和配置路径
ckpt_converter = 'checkpoints/converter'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = '/content/converted_audio'
os.makedirs(output_dir, exist_ok=True)

# 初始化音色转换器
tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

# 提取原始音色
source_se, _ = se_extractor.get_se('/content/MeloTTS/combined_tts_audio.mp3', tone_color_converter, vad=True)
# 目标音色设为TTS音频的音色
# 如果TTS音频的音色不需要特别提取，这行代码可以省略
target_se, _ = se_extractor.get_se(audio_file_path, tone_color_converter, vad=True)

# 对每对音频文件进行处理
for (begin_time, end_time, original_path), (begin_time, end_time, tts_path) in zip(origin_audio_segments, tts_audio_segments):

    # 设置输出文件路径
    output_path = os.path.join(output_dir, os.path.basename(tts_path))

    # 应用音色转换
    tone_color_converter.convert(
        audio_src_path=tts_path,
        src_se=source_se,
        tgt_se=target_se,  # 如果TTS音频不提取音色，这里使用source_se代替
        output_path=output_path,
        message="@MyShell"  # 根据需要修改或省略该参数
    )

    print(f"Converted audio saved to: {output_path}")


Loaded checkpoint 'checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []
Converted audio saved to: /content/converted_audio/0.0-5.4.mp3
Audio too short, fail to add watermark
Converted audio saved to: /content/converted_audio/5.4-5.9.mp3
Converted audio saved to: /content/converted_audio/5.9-8.4.mp3
Converted audio saved to: /content/converted_audio/8.4-11.4.mp3
Converted audio saved to: /content/converted_audio/11.4-16.1.mp3
Converted audio saved to: /content/converted_audio/16.1-19.5.mp3
Converted audio saved to: /content/converted_audio/19.5-26.0.mp3


In [ ]:
!pip install moviepy

In [ ]:
from moviepy.editor import VideoFileClip

# Path to your original video
original_video_path = '/content/Building domain-specific compilers quickly with MLIR compiler infrastructure  Chris Lattner.mp4'
# Path where the trimmed video will be saved
trimmed_video_path = '/content/trimmed_video.mp4'

# Load the original video
video_clip = VideoFileClip(original_video_path)

# Trim the first 30 seconds
# Here, we assume the video is longer than 30 seconds
trimmed_clip = video_clip.subclip(0, 30)

# Write the trimmed clip to a new video file
trimmed_clip.write_videofile(trimmed_video_path, codec='libx264', audio_codec='aac')

# Cleanup
video_clip.close()
trimmed_clip.close()


t:   0%|          | 2/900 [00:14<1:51:14,  7.43s/it, now=None]

Moviepy - Building video /content/trimmed_video.mp4.
MoviePy - Writing audio in trimmed_videoTEMP_MPY_wvf_snd.mp4



t:   0%|          | 2/900 [00:16<2:00:09,  8.03s/it, now=None]

MoviePy - Done.
Moviepy - Writing video /content/trimmed_video.mp4




t:   0%|          | 2/900 [01:17<9:41:02, 38.82s/it, now=None]

Moviepy - Done !
Moviepy - video ready /content/trimmed_video.mp4


In [ ]:
from moviepy.editor import VideoFileClip, concatenate_audioclips, AudioFileClip, CompositeAudioClip
from moviepy.audio.AudioClip import AudioClip
import os

video_file_path = '/content/trimmed_video.mp4'
audio_segments_dir = '/content/converted_audio'  # Directory containing your audio segments
output_video_path = '/content/new_video_with_audio.mp4'  # Output video file path

# Load the original video
video = VideoFileClip(video_file_path)

# Generate a silent audio clip of the same duration as the video
silent_audio = AudioClip(lambda t: 0, duration=video.duration).set_fps(44100)

audio_clips = []  # Initialize an empty list to store audio clips with their start times

# Iterate over audio segment files in the directory
for segment_file in sorted(os.listdir(audio_segments_dir)):
    if segment_file.endswith('.mp3') or segment_file.endswith('.wav'):
        # Parse start and end times from the file name
        file_name = os.path.splitext(segment_file)[0]
        start_time, end_time = map(float, file_name.split('-'))

        # Load the audio segment
        segment = AudioFileClip(os.path.join(audio_segments_dir, segment_file)).set_start(start_time)

        # Add the audio segment to the list
        audio_clips.append(segment)

# Combine silent audio with the segments
if audio_clips:
    final_audio = CompositeAudioClip([silent_audio] + audio_clips)
else:
    final_audio = silent_audio

# Set the composite audio as the audio of the video clip
video_with_audio = video.set_audio(final_audio)

# Export the video with the new audio
video_with_audio.write_videofile(output_video_path, codec='libx264', audio_codec='aac')

# Cleanup
video.close()
video_with_audio.close()
for clip in audio_clips:
    clip.close()


t:   0%|          | 2/900 [01:54<14:19:59, 57.46s/it, now=None]

Moviepy - Building video /content/new_video_with_audio.mp4.
MoviePy - Writing audio in new_video_with_audioTEMP_MPY_wvf_snd.mp4



t:   0%|          | 2/900 [01:55<14:27:11, 57.94s/it, now=None]

MoviePy - Done.
Moviepy - Writing video /content/new_video_with_audio.mp4




t: 100%|█████████▉| 899/901 [00:47<00:00, 21.31it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/trimmed_video.mp4, 2764800 bytes wanted but 0 bytes read,at frame 900/901, at time 30.00/30.02 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+


t:   0%|          | 2/900 [02:45<20:36:57, 82.65s/it, now=None]

Moviepy - Done !
Moviepy - video ready /content/new_video_with_audio.mp4


In [ ]:
from IPython.display import Video

# 视频文件的路径
video_file_path = '/content/new_video_with_audio.mp4'

# 在Colab中显示视频
Video(video_file_path)
